In [134]:
import pandas as pd
import serviceWrapperAsync as wrapper
import serviceHandlers as handler
from serviceWrapperAsync import NamSorEndpoint as endpoint
import wrapperManager as wrapman
import evaluator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
# Getting a dataset

df = pd.read_csv("../../data/all.csv")
df = df[(df['hasMiddleName']==False) & (df['firstName'].notnull())]
df = df.sample(5)
df

,Unnamed: 0,fullName,firstName,lastName,gender,isoCountry,continent,birthYear,hasMiddleName,hasNoLastName,source
6470,6470,KOXHAKU Grisi,Grisi,KOXHAKU,male,AL,Europe,2006,False,False,kaggleOlympic
8945,8945,JI Yuchan,Yuchan,JI,male,KP,Asia,2002,False,False,kaggleOlympic
3558,3558,Pyotr Buslov,Pyotr,Buslov,male,RU,Europe,1976,False,False,wikidata
6016,6016,WILLIAMS Ashley,Ashley,WILLIAMS,female,JM,North America,1996,False,False,kaggleOlympic
15649,15649,BELAC Ana,Ana,BELAC,female,SI,Europe,1997,False,False,kaggleOlympic


# Testing synchronous wrapper

## Testing GenderizeIoHandler

In [136]:
# genderizeIO_handler = handlers.GenderizeIoHandler(datasource=df, hasSubscription=False)
# genderizeIO_handler.get_prediction(True)

## Testing GenderAPI

In [137]:
# genderAPI_IO_Handler = handlers.GenderAPI_IO_Handler(datasource=df)

# genderAPI_IO_Handler.callAPI(useLocalization=True)

## Testing GenderGuesser

In [138]:
# genderGuesserHandler = handler.GenderGuesserHandler(df)

# genderGuesserHandler.get_prediction()

## Testing GenderAPI.com 

In [139]:
# genderAPI_com_handler = handlers.GenderAPI_com_Handler(df)

# genderAPI_com_handler.get_prediction(True, True)

## Testing NamSor Handler

In [140]:
# namSor_handler = handlers.NamSorHandler(df)

# namSor_handler.get_prediction(endpoint=NamSorEndpoint.FULL_NAME)

## Testing NameAPI

In [141]:
# nameAPI_handler = handler.NameAPIHandler(df)

# await nameAPI_handler.get_prediction(True)

# Testing async Wrapper

In [142]:
genderize_io_wrapper = wrapper.GenderizeWrapper(df)
nameAPI_wrapper = wrapper.NameAPIWrapper(df)
namSor_wrapper = wrapper.NamSorWrapper(df)
genderCom_wrapper = wrapper.GenderAPI_com_Wrapper(df)
genderIO_wrapper = wrapper.GenderAPI_IO_Wrapper(df)

In [144]:
testign_result = await nameAPI_wrapper.get_prediction_async(useFullName=False)

In [145]:
testign_result

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraPreciseGenderPredicted,extraMaleProportion,extraConfidence
0,6470,KOXHAKU Grisi,Grisi,male,unknown,AL,False,NameAPI,UNKNOWN,None,0.8
1,8945,JI Yuchan,Yuchan,male,unknown,KP,False,NameAPI,UNKNOWN,None,0.8
2,3558,Pyotr Buslov,Pyotr,male,unknown,RU,False,NameAPI,UNKNOWN,None,0.8
3,6016,WILLIAMS Ashley,Ashley,female,unknown,JM,False,NameAPI,UNKNOWN,None,0.8
4,15649,BELAC Ana,Ana,female,unknown,SI,False,NameAPI,UNKNOWN,None,0.8


# testing wrapper manager

In [ ]:
testing_manager = wrapman.WrapperManager(df)

testign_result = await testing_manager.run_all(config={
            "genderize": {"useLocalization": False},
            "genderapi_io": {"useLocalization": False, "useAI": False},
            "genderapi_com": {"useFullName": False, "useLocalization": False},
            "namsor": {"endpoint": endpoint.FIRST_NAME},
            "nameapi": {"useFullName": False}
        })

In [151]:
testign_result['nameapi']

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraPreciseGenderPredicted,extraMaleProportion,extraConfidence
0,6470,KOXHAKU Grisi,Grisi,male,unknown,AL,False,NameAPI,UNKNOWN,None,0.8
1,8945,JI Yuchan,Yuchan,male,unknown,KP,False,NameAPI,UNKNOWN,None,0.8
2,3558,Pyotr Buslov,Pyotr,male,unknown,RU,False,NameAPI,UNKNOWN,None,0.8
3,6016,WILLIAMS Ashley,Ashley,female,unknown,JM,False,NameAPI,UNKNOWN,None,0.8
4,15649,BELAC Ana,Ana,female,unknown,SI,False,NameAPI,UNKNOWN,None,0.8


In [152]:
manager = wrapman.WrapperManager(df)
results = await manager.run_subset(
            ["genderize", "namsor"],
            config={"genderize": {"useLocalization": True}, "namsor": {"endpoint": endpoint.FIRST_NAME_GEO}}
        )

In [153]:
resultGenderize = results['genderize']
resultNamSor = results['namsor']

sameCols = [col for col in resultNamSor.columns if col in resultGenderize.columns]
print(sameCols)

['index', 'fullName', 'namePassed', 'correctGender', 'predictedGender', 'localization', 'useLocalization', 'serviceUsed']


In [154]:
testConcat = pd.concat([resultGenderize[sameCols], resultNamSor[sameCols]])
testConcat

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed
0,6470,KOXHAKU Grisi,Grisi,male,female,AL,True,genderize.IO
1,8945,JI Yuchan,Yuchan,male,female,KP,True,genderize.IO
2,3558,Pyotr Buslov,Pyotr,male,male,RU,True,genderize.IO
3,6016,WILLIAMS Ashley,Ashley,female,female,JM,True,genderize.IO
4,15649,BELAC Ana,Ana,female,female,SI,True,genderize.IO
0,6470,KOXHAKU Grisi,Grisi,male,female,AL,True,NamSor
1,8945,JI Yuchan,Yuchan,male,female,KP,True,NamSor
2,3558,Pyotr Buslov,Pyotr,male,male,RU,True,NamSor
3,6016,WILLIAMS Ashley,Ashley,female,female,JM,True,NamSor
4,15649,BELAC Ana,Ana,female,female,SI,True,NamSor


# Testing the Evaluator

In [155]:
eva = evaluator.Evaluator(testConcat[testConcat['serviceUsed']=='genderize.IO'])

In [156]:
testConcat[testConcat['serviceUsed']=='genderize.IO']

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed
0,6470,KOXHAKU Grisi,Grisi,male,female,AL,True,genderize.IO
1,8945,JI Yuchan,Yuchan,male,female,KP,True,genderize.IO
2,3558,Pyotr Buslov,Pyotr,male,male,RU,True,genderize.IO
3,6016,WILLIAMS Ashley,Ashley,female,female,JM,True,genderize.IO
4,15649,BELAC Ana,Ana,female,female,SI,True,genderize.IO


In [157]:
conf_matrix = eva.get_confusion_matrix(None)

In [158]:
conf_matrix

,f_pred,m_pred,u_pred
f,2,0,0
m,2,1,0
u,0,0,0


In [159]:
eva.compute_all_errors(conf_matrix)

{'error_with_unknown': np.float64(0.4),
 'error_without_unknown': np.float64(0.4),
 'error_unknown': np.float64(0.0),
 'error_gender_bias': np.float64(0.4),
 'weighted_error': np.float64(0.4)}